In [1]:
import pickle
import random
from simulation_utils import *
import numpy as np
import torch
import scipy.sparse as sp


In [2]:
dataset = "Flickr"
data = load_sim_debug(dataset)["train_outputs"]
data.keys()

dict_keys(['adj', 'health_condition', 'action_dosage', 'avg_neighbor_action_dosage', 'action_application_point', 'avg_neighbor_action_application', 'treatment_probs', 'noise_terms', 'mean', 'std'])

In [3]:
health_condition = data["health_condition"]
action_dosage = data["action_dosage"]
avg_neighbor_action_dosage = data["avg_neighbor_action_dosage"]
mean = data["mean"]
std = data["std"]

In [4]:
features = ["health_condition", "action_dosage", "avg_neighbor_action_dosage"]
for k in features:
    data[k+"_norm"] = (data[k] - mean[k]) / std[k]

In [5]:
data.keys()

dict_keys(['adj', 'health_condition', 'action_dosage', 'avg_neighbor_action_dosage', 'action_application_point', 'avg_neighbor_action_application', 'treatment_probs', 'noise_terms', 'mean', 'std', 'health_condition_norm', 'action_dosage_norm', 'avg_neighbor_action_dosage_norm'])

In [6]:
# save_sim(dataset,data,norm=True)

In [7]:
adj = data["adj"]

In [8]:
adj = adj + np.multiply(adj.T,(adj.T > adj)) - np.multiply(adj,adj.T > adj)
adj+=np.identity(adj.shape[0],dtype=int)

In [9]:
adj = adj.astype(float)

In [10]:
adj

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [11]:
def normalize(mx):
    """
    Row-normalize sparse matrix 
    
    code from https://github.com/tkipf/pygcn
    """
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [12]:
normalize(adj)

array([[0.03225806, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05263158, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.07142857, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.33333333, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.5       ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.5       ]])